# Imports

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Input
from tensorflow.keras.layers import Conv2D, BatchNormalization, MaxPool2D, ReLU, Flatten, Dense
tf.random.set_seed(666)

In [2]:
# Load the training and testing set of CIFAR10
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.cifar10.load_data()

X_train = X_train.astype('float32')
X_train = X_train/255.

X_test = X_test.astype('float32')
X_test = X_test/255.

y_train = tf.reshape(tf.one_hot(y_train, 10), shape=(-1, 10))
y_test = tf.reshape(tf.one_hot(y_test, 10), shape=(-1, 10))

170500096/170498071 [==============================] - 6s 0us/step


In [8]:
# Create TensorFlow dataset
BATCH_SIZE = 256
AUTOTUNE = tf.data.experimental.AUTOTUNE

train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train))
train_ds = train_ds.shuffle(1024).cache().batch(BATCH_SIZE).prefetch(AUTOTUNE)

test_ds = tf.data.Dataset.from_tensor_slices((X_test, y_test))
test_ds = test_ds.cache().batch(BATCH_SIZE).prefetch(AUTOTUNE)

In [14]:
for image, label in train_ds.take(1):
  print(image.shape)
  print(label.shape)

(256, 32, 32, 3)
(256, 10)


In [13]:
class Organism:
    def __init__(self, chromosome, phase):
        self.phase = phase
        self.chromosome = chromosome
    def build_model(self):
        if self.phase == 0:
            keras.backend.clear_session()
            
            inputs = Input(shape=(32,32,3))
            x = Conv2D(filters=self.chromosome['a_output_channels'],
                        kernel_size=self.chromosome['a_filter_size'],
                        use_bias=self.chromosome['a_include_BN'])(inputs)
            if self.chromosome['a_include_BN']:
                x = BatchNormalization()(x)
            x = self.chromosome['activation_type']()(x)
            if self.chromosome['include_pool']:
                x = self.chromosome['pool_type'](x)

            x = Conv2D(filters=self.chromosome['b_output_channels'],
                        kernel_size=self.chromosome['b_filter_size'],
                        use_bias=self.chromosome['b_include_BN'])(x)
            if self.chromosome['b_include_BN']:
                x = BatchNormalization()(x)
            x = self.chromosome['activation_type']()(x)

            x = Flatten()(x)
            x = Dense(10, activation='softmax')(x)

            self.model = tf.keras.Model(inputs=[inputs], outputs=[x])
            self.model.compile(optimizer='adam',
                                loss='categorical_crossentropy',
                                metrics=['accuracy'])
        else:
            print('Phase under construction')
    
    def fitnessFunction(self, train_ds, test_ds):
        self.model.fit(train_ds,
                       epochs=5,
                       verbose=1)
        _, self.fitness = self.model.evaluate(test_ds, verbose=0)
    
    def crossover(self,partner):
        pass
    
    def mutation(self, mutationRate):
        pass

In [10]:
demo = Organism({'a_output_channels':64,
                 'b_output_channels':128,
                 'a_filter_size':(3,3),
                 'b_filter_size':(3,3),
                 'a_include_BN':True,
                 'b_include_BN':True,
                 'activation_type': ReLU,
                 'include_pool': True,
                 'pool_type': MaxPool2D()
                 }, 
                phase=0)
keras.backend.clear_session()
demo.build_model()
# tf.keras.utils.plot_model(demo.model, show_shapes=True)

In [11]:
demo.fitnessFunction(train_ds, test_ds)

Epoch 1/10
196/196 [==============================] - 2s 11ms/step - loss: 1.5339 - accuracy: 0.5197
Epoch 2/10
196/196 [==============================] - 2s 10ms/step - loss: 1.0066 - accuracy: 0.6587
Epoch 3/10
196/196 [==============================] - 2s 10ms/step - loss: 0.8354 - accuracy: 0.7158
Epoch 4/10
196/196 [==============================] - 2s 10ms/step - loss: 0.7306 - accuracy: 0.7490
Epoch 5/10
196/196 [==============================] - 2s 10ms/step - loss: 0.6492 - accuracy: 0.7776
Epoch 6/10
196/196 [==============================] - 2s 10ms/step - loss: 0.6136 - accuracy: 0.7868
Epoch 7/10
196/196 [==============================] - 2s 10ms/step - loss: 0.5536 - accuracy: 0.8058
Epoch 8/10
196/196 [==============================] - 2s 10ms/step - loss: 0.4825 - accuracy: 0.8330
Epoch 9/10
196/196 [==============================] - 2s 10ms/step - loss: 0.4342 - accuracy: 0.8510
Epoch 10/10
196/196 [==============================] - 2s 10ms/step - loss: 0.4198 - accura

In [12]:
demo.fitness

0.6377999782562256